# Machine Learning - Airline Delay Prediction Algorithm

## Imports

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Adding the root directory of the project to the Python path, so that it will also look here for modules

import config

import pandas as pd

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta



## import data

In [2]:
from config import RAW_AIRLINE_DELAY_CAUSE
from config import RAW_AIRLINES 
from config import RAW_AIRPORTS
from config import RAW_DATA_FYYTg
from config import RAW_FLIGHTS
from config import CLEANED_PLANES
from config import CLEANED_AIRCRAFT_DATA


df_airline_delay_cause = pd.read_csv(RAW_AIRLINE_DELAY_CAUSE)

df_airlines = pd.read_csv(RAW_AIRLINES)

df_airports= pd.read_csv(RAW_AIRPORTS)

df_cleaned_aircraft_data = pd.read_csv(CLEANED_AIRCRAFT_DATA)

df_flights = pd.read_csv(RAW_FLIGHTS)

df_planes_cleaned = pd.read_csv(CLEANED_PLANES)

/var/folders/kl/2y01c7y51cj42bdww37cvc5c0000gn/T/ipykernel_86853/219988536.py:18: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv(RAW_FLIGHTS)


## Observe columns in common

In [3]:
# function to display the columns of a df

def display_columns(df_name, df):
    print(f"Columns of {df_name}:")
    print(df.columns)
    print()

# dictionary to store the variable names and corresponding dfs

dfs = {
    'AIRLINE DELAY CAUSE': df_airline_delay_cause,
    'AIRLINES': df_airlines, 
    'AIRPORTS': df_airports, 
    'CLEANED AIRCRAFT DATA':df_cleaned_aircraft_data,
    'FLIGHTS': df_flights,
    'CLEANED PLANES':df_planes_cleaned
    }

for df_name, df in dfs.items():
    display_columns(df_name, df)

Columns of AIRLINE DELAY CAUSE:
Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

Columns of AIRLINES:
Index(['IATA_CODE', 'AIRLINE'], dtype='object')

Columns of AIRPORTS:
Index(['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

Columns of CLEANED AIRCRAFT DATA:
Index(['tailnum', 'year', 'type', 'manufacturer', 'model', 'engine_count',
       'engine_type'],
      dtype='object')

Columns of FLIGHTS:
Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
      

In [4]:
# dictionary to store the variable names and corresponding dfs

dfs_2 = {
    'AIRLINE DELAY CAUSE': df_airline_delay_cause,
    'AIRLINES': df_airlines, 
    'AIRPORTS': df_airports, 
    'CLEANED AIRCRAFT DATA':df_cleaned_aircraft_data,
    'FLIGHTS': df_flights,
    'CLEANED PLANES':df_planes_cleaned,
    }

for df_name, df in dfs_2.items():
    print(f'Columns of {df_name}:')
    print(df.columns)
    print()

Columns of AIRLINE DELAY CAUSE:
Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

Columns of AIRLINES:
Index(['IATA_CODE', 'AIRLINE'], dtype='object')

Columns of AIRPORTS:
Index(['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

Columns of CLEANED AIRCRAFT DATA:
Index(['tailnum', 'year', 'type', 'manufacturer', 'model', 'engine_count',
       'engine_type'],
      dtype='object')

Columns of FLIGHTS:
Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
      

In [3]:
pd.set_option('display.max_rows', None)


In [10]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [8]:
df_flights.sample(20)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4354908,2015,9,29,2,WN,2164,N954WN,MHT,MCO,645,643.0,-2.0,8.0,651.0,195.0,184.0,163.0,1142,934.0,13.0,1000,947.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3037842,2015,7,10,5,AA,281,N634AA,MIA,DFW,600,557.0,-3.0,16.0,613.0,186.0,178.0,158.0,1121,751.0,4.0,806,755.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2428223,2015,6,3,3,OO,4841,N602XJ,MSP,GFK,1354,1353.0,-1.0,32.0,1425.0,75.0,87.0,51.0,284,1516.0,4.0,1509,1520.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2323135,2015,5,28,4,AA,1619,N3KAAA,BOS,MIA,630,621.0,-9.0,20.0,641.0,212.0,193.0,168.0,1258,929.0,5.0,1002,934.0,-28.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4380075,2015,9,30,3,AA,1823,N203UW,PHL,PHX,1555,1554.0,-1.0,19.0,1613.0,312.0,298.0,271.0,2075,1744.0,8.0,1807,1752.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3413749,2015,8,1,6,EV,2633,N907EV,DFW,LRD,850,851.0,1.0,18.0,909.0,85.0,84.0,63.0,396,1012.0,3.0,1015,1015.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1358085,2015,3,29,7,UA,1719,N64809,DEN,MCO,955,1052.0,57.0,16.0,1108.0,210.0,193.0,171.0,1546,1559.0,6.0,1525,1605.0,40.0,0,0,NaN,0.0,0.0,4.0,36.0,0.0
1178569,2015,3,18,3,EV,3937,N14991,IAH,MLU,910,904.0,-6.0,25.0,929.0,63.0,71.0,38.0,262,1007.0,8.0,1013,1015.0,2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2226295,2015,5,21,4,WN,279,N211WN,PHX,SAT,1910,1949.0,39.0,5.0,1954.0,125.0,113.0,104.0,843,2338.0,4.0,2315,2342.0,27.0,0,0,NaN,0.0,0.0,0.0,27.0,0.0
1694275,2015,4,19,7,MQ,3212,N845MQ,DBQ,ORD,645,640.0,-5.0,8.0,648.0,63.0,58.0,31.0,147,719.0,19.0,748,738.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pd.set_option('display.max_columns', None)


In [6]:
df_airline_delay_cause.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,6,YV,Mesa Airlines Inc.,BDL,"Hartford, CT: Bradley International",73.0,9.0,5.50,0.00,1.07,0.0,2.44,0.0,0.0,664.0,239.0,0.0,39.0,0.0,386.0
1,2024,6,YV,Mesa Airlines Inc.,BHM,"Birmingham, AL: Birmingham-Shuttlesworth Inter...",95.0,41.0,14.96,2.26,11.41,0.0,12.37,2.0,0.0,3232.0,685.0,260.0,389.0,0.0,1898.0
2,2024,6,YV,Mesa Airlines Inc.,BNA,"Nashville, TN: Nashville International",75.0,14.0,6.85,1.00,1.77,0.0,4.37,2.0,0.0,1297.0,807.0,23.0,70.0,0.0,397.0
3,2024,6,YV,Mesa Airlines Inc.,BRO,"Brownsville, TX: Brownsville South Padre Islan...",21.0,4.0,1.88,1.00,0.12,0.0,1.00,0.0,0.0,157.0,50.0,25.0,3.0,0.0,79.0
4,2024,6,YV,Mesa Airlines Inc.,BTR,"Baton Rouge, LA: Baton Rouge Metropolitan/Ryan...",83.0,27.0,15.15,0.43,2.23,0.0,9.19,2.0,0.0,1972.0,939.0,9.0,147.0,0.0,877.0


## Verify if possible to merge df_flights and df_airlines on 'IATA_CODE'

In [16]:
f_airline = df_flights['AIRLINE'].head(10)
a_airline = df_airlines['IATA_CODE'].head(10)

f_airline = f_airline.to_frame()
f_airline['a_airline'] = a_airline

f_airline

,AIRLINE,a_airline
0,AS,UA
1,AA,AA
2,US,US
3,AA,F9
4,AS,B6
5,DL,OO
6,NK,AS
7,US,NK
8,AA,WN
9,DL,DL


In [19]:
df_flights['AIRLINE'].sort_values().value_counts()


AIRLINE
WN    1261855
DL     875881
AA     725984
OO     588353
EV     571977
UA     515723
MQ     294632
B6     267048
US     198715
AS     172521
NK     117379
F9      90836
HA      76272
VX      61903
Name: count, dtype: int64

In [20]:
df_airlines['IATA_CODE'].head(10).sort_values().value_counts()

IATA_CODE
AA    1
AS    1
B6    1
DL    1
F9    1
NK    1
OO    1
UA    1
US    1
WN    1
Name: count, dtype: int64

In [ ]:
df_airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


In [28]:
df_merged = pd.merge(df_flights, df_airlines, left_on='AIRLINE', right_on='IATA_CODE', how='outer', suffixes=('_flights', '_airlines'))

In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [29]:
df_merged.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_flights,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRLINE_airlines
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
1,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
2,2015,1,1,4,AS,108,N309AS,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,194.0,173.0,1448,451.0,4.0,509,455.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
3,2015,1,1,4,AS,122,N413AS,ANC,PDX,50,46.0,-4.0,11.0,57.0,215.0,201.0,187.0,1542,504.0,3.0,525,507.0,-18.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
4,2015,1,1,4,AS,130,N457AS,FAI,SEA,115,107.0,-8.0,25.0,132.0,213.0,218.0,186.0,1533,538.0,7.0,548,545.0,-3.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.


## Verify compatibility of merging df_flights['ORIGIN_AIRPORT'] and df_airports['IATA_CODE] 
## Verify compatibility of merging df_flights['DESTINATION_AIRPORT'] and df_airports['IATA_CODE] 


In [ ]:
df_flights['AIRLINE'].sort_values().value_counts()


f_o_airport = df_flights['ORIGIN_AIRPORT'].astype(str).sort_values().value_counts()
f_d_airport = df_flights['DESTINATION_AIRPORT'].astype(str).sort_values().value_counts()
a_airport = df_airports['IATA_CODE'].astype(str).sort_values().value_counts()

print(f"flights origin airport:\n{f_o_airport}\n")
print(f"flights destination airport:\n{f_d_airport}\n")
print(f"airport iata code:\n{a_airport}\n")


flights origin airport:
ORIGIN_AIRPORT
ATL      346836
ORD      285884
DFW      239551
DEN      196055
LAX      194673
SFO      148008
PHX      146815
IAH      146622
LAS      133181
MSP      112117
MCO      110982
SEA      110899
DTW      108500
BOS      107847
EWR      101772
CLT      100324
LGA       99605
SLC       97210
JFK       93811
BWI       86079
MDW       80886
DCA       73794
FLL       73624
SAN       70207
MIA       69341
PHL       66021
TPA       63906
DAL       59699
HOU       52042
BNA       47580
PDX       47085
STL       46963
HNL       43172
OAK       42316
AUS       42067
MSY       38804
MCI       38665
SJC       38200
SMF       37212
SNA       37187
CLE       34589
IAD       34305
10397     32588
RDU       31896
MKE       29825
SAT       29788
13930     27652
RSW       27350
IND       25558
SJU       24656
CMH       24187
PIT       24118
PBI       22573
11298     21044
OGG       20627
CVG       20031
ABQ       19192
BUR       18889
BDL       18799
11292     18136
J

# old code

In [42]:
import sys
import os

# Adjust the path based on Step 1's output
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
print(f"Adding to sys.path: {src_path}")  # Debugging step

sys.path.append(src_path)

Adding to sys.path: /Users/kimberlyblack/Documents/GITHUB-projects/WCS-Project-4/WCS-Project_Late-Aircraft-Prediction-System_Dashboard_Customer-Review-Sentiment-Analysis/src


In [43]:
import sys
print(sys.path)

['/Users/kimberlyblack/Documents/GITHUB-projects/WCS-Project-4/WCS-Project_Late-Aircraft-Prediction-System_Dashboard_Customer-Review-Sentiment-Analysis/notebooks', '/opt/anaconda3/lib/python311.zip', '/opt/anaconda3/lib/python3.11', '/opt/anaconda3/lib/python3.11/lib-dynload', '', '/opt/anaconda3/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/kimberlyblack/Documents/GITHUB-projects/WCS-Project-4/WCS-Project_Late-Aircraft-Prediction-System_Dashboard_Customer-Review-Sentiment-Analysis', '/Users/kimberlyblack/Documents/GITHUB-projects/WCS-Project-4/WCS-Project_Late-Aircraft-Prediction-System_Dashboard_Customer-Review-Sentiment-Analysis/src']


In [44]:
from f_outliers_heatmap_df import analyze_column_uniqueness, detect_and_visualize_outliers

# Run analysis
#analyze_column_uniqueness(df_flights)

In [7]:
detect_and_visualize_outliers(df_flights)


Detecting outliers for numeric columns:

Outliers Summary:
                  YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
Number of values   0.0    0.0  0.0          0.0   27073.000000   
% of values        0.0    0.0  0.0          0.0       0.465245   

                  SCHEDULED_DEPARTURE  DEPARTURE_TIME  DEPARTURE_DELAY  \
Number of values                  0.0             0.0    736242.000000   
% of values                       0.0             0.0        12.652208   

                       TAXI_OUT  WHEELS_OFF  ...  SCHEDULED_ARRIVAL  \
Number of values  282602.000000         0.0  ...                0.0   
% of values            4.856473         0.0  ...                0.0   

                  ARRIVAL_TIME  ARRIVAL_DELAY      DIVERTED     CANCELLED  \
Number of values           0.0  512002.000000  15187.000000  89884.000000   
% of values                0.0       8.798678      0.260986      1.544643   

                  AIR_SYSTEM_DELAY  SECURITY_DELAY  AIRLINE_DELAY  \
Num

: 

In [45]:
detect_and_visualize_outliers(df_merged)


Detecting outliers for numeric columns:

Outliers Summary:
                  YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
Number of values   0.0    0.0  0.0          0.0   27073.000000   
% of values        0.0    0.0  0.0          0.0       0.465245   

                  SCHEDULED_DEPARTURE  DEPARTURE_TIME  DEPARTURE_DELAY  \
Number of values                  0.0             0.0    736242.000000   
% of values                       0.0             0.0        12.652208   

                       TAXI_OUT  WHEELS_OFF  SCHEDULED_TIME   ELAPSED_TIME  \
Number of values  282602.000000         0.0   299011.000000  291084.000000   
% of values            4.856473         0.0        5.138459       5.002235   

                       AIR_TIME       DISTANCE  WHEELS_ON        TAXI_IN  \
Number of values  296342.000000  349511.000000        0.0  282538.000000   
% of values            5.092593       6.006294        0.0       4.855373   

                  SCHEDULED_ARRIVAL  ARRIVAL_TIME  ARR

: 